In [1]:
from LinearTrainer import GridSearcher
from LinearTrainer import LinearTrainer

In [ ]:
from datasets.utils_datasets import get_mnist_binary_static_loaders
seed = 5
loaders =  get_mnist_binary_static_loaders(seed=seed)

In [2]:
gr = GridSearcher(loaders)

In [3]:
path = 'experiment_1'

In [3]:
gr.conduct_experiment(path=path)

Beta-VAE
conducting experiment 1 with beta: 0.010000000000000005 and learning rate: 0.01
using seed: 1
using seed: 10
conducting experiment 1 with beta: 10.000000000000002 and learning rate: 0.01
using seed: 1
using seed: 10
MR-VAE
conducting experiment 1 and learning rate: 0.01


In [ ]:
betas = gr.betas

In [4]:
import pandas as pd

# BETA

In [7]:
df_beta_vae : pd.DataFrame = pd.read_csv(f'{path}/beta_vae.csv', index_col=0)

In [8]:
df_beta_vae.head()

,use_multi_rate,mean_loss,beta,lr
0,False,886.745796,0.01,0.01
1,False,1340.995643,10.00,0.01


In [26]:
# select the best for beta 
df_beta_vae_sorted = df_beta_vae.sort_values(by=['mean_loss']).groupby(by=['lr', 'beta']).head(1)

In [27]:
df_beta_vae_sorted

,use_multi_rate,mean_loss,beta,lr
0,False,886.745796,0.01,0.01
1,False,1340.995643,10.00,0.01


In [30]:
import torch
torch.manual_seed(989)
rec_beta_vae = []
distortion_beta_vae = []
for i in range(0, len(df_beta_vae_sorted)):
    beta = df_beta_vae_sorted.at[i, 'beta']
    lr = df_beta_vae_sorted.at[i, 'lr']
    print(f'best learning rate for beta: {beta} is lr: {lr}\n')
    trainer = LinearTrainer(loaders, use_multi_rate=False, beta=beta, lr=lr)
    trainer.train()
    _, (rec_losses, kdl_losses) = trainer.rate_distortion_curve_value(beta, beta)
    rec_beta_vae.append(rec_losses)
    distortion_beta_vae.append(kdl_losses)


best learning rate for beta: 0.01 is lr: 0.01

best learning rate for beta: 10.000000000000002 is lr: 0.01


# MULTI-RATE VAE

In [33]:
df_multi_rate : pd.DataFrame = pd.read_csv(f'{path}/mr_vae.csv', index_col=0)


In [34]:
df_multi_rate.head()

,use_multi_rate,mean_loss,beta,lr
0,True,1453.67288,1.0,0.01


In [36]:
best_lr = df_multi_rate.sort_values(by=['mean_loss']).at[0, 'lr']
print(f'best lr for multi-rate vae  is {best_lr}')

best lr for multi-rate vae  is 0.01


In [ ]:
rec_mr_vae = []
distortion_mr_vae = []
for b in betas:
    trainer = LinearTrainer(loaders, use_multi_rate=True, lr=best_lr)
    trainer.train()
    _, (rec_losses, kdl_losses) = trainer.rate_distortion_curve_value(b, 1.)
    rec_mr_vae.append(rec_losses)
    distortion_mr_vae.append(kdl_losses)